# 创建负责任 AI 仪表板以评估模型

比较和评估机器学习模型时，需要查看的不仅仅是其性能指标。 借助 Azure 机器学习可以创建负责任 AI 仪表板，以探索模型在不同数据队列中的效果。 

## 准备数据

要创建负责的 AI 仪表板，需要训练和测试数据集，该数据集存储为 Parquet 文件，并注册为数据资产。

数据当前存储为 CSV 文件。 让我们将其转换为 Parquet 文件。 


In [ ]:
## 在继续之前

需要最新版本的 azureml-ai-ml 包才能运行此笔记本中的代码。 运行下面的单元以验证是否已安装它。

> **注意**：
> 如果未安装 azure-ai-ml 包，请运行 `pip install azure-ai-ml` 以进行安装。

In [ ]:
## 连接到工作区

安装必需的 SDK 包后，就可以连接到工作区了。

若要连接到工作区，我们需要标识符参数 - 订阅 ID、资源组名称和工作区名称。 已为你填写资源组名称和工作区名称。 只需订阅 ID 即可完成命令。

若要查找所需的参数，请单击工作室右上角的订阅和工作区名称。 右侧将打开一个窗格。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 复制订阅 ID，并将“YOUR-SUBSCRIPTION-ID”替换为复制的值。 </p>

## 创建数据资产

要创建负责任 AI 仪表板，需要将训练和测试数据集注册为 MLtable 数据资产。 MLtable 数据资产引用之前创建的 Parquet 文件。

In [ ]:
## 生成管道以创建负责任 AI 仪表板

若要创建仪表板，你将生成一个管道，其中包含默认在 Azure 机器学习工作区中注册的预生成组件。

### 注册模型

已为你训练机器学习模型。 该模型预测患者是否患有糖尿病。 所有模型文件都存储在 `model` 文件夹中。 

通过指向 `model` 文件夹及其内容注册模型。

In [ ]:
## 构建管道

要创建负责任 AI 仪表板，需要使用预生成的组件创建一个管道。 可以选择要使用的组件以及要包含在仪表板中的功能。 你将创建一个包含错误分析和模型可解释性的仪表板。 

除了负责任 AI 功能之外，生成仪表板的管道还需要在开始时包含一个组件用于构造仪表板，并在最后包含一个组件用于收集所有生成的见解。

In [ ]:
检索到所有要使用的组件后，可以生成管道并按适当的顺序连接组件：

1. 构造仪表板。
1. 添加错误分析。
1. 添加说明。
1. 收集所有见解并在仪表板中将其可视化。

现在，管道已经生成，需要定义两个必要的输入：训练和测试数据集。

In [ ]:
最后，我们将所有内容放在一起：将输入分配给管道，并设置目标列（预测标签）。

In [ ]:
## 运行管道

成功生成管道后，可以将其提交。 以下代码将提交管道并检查管道的状态。 还可以在工作室中查看管道的状态。 

管道完成后，可以在工作室中查看仪表板。 

In [ ]:
# Get handle to azureml registry for the RAI built in components
registry_name = "azureml"
ml_client_registry = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    registry_name=registry_name,
)
print(ml_client_registry)

### Register the model

A machine learning model has already been trained for you. The model predicts whether a patient has diabetes. All model files are stored in the `model` folder. 

Register the model by pointing to the `model` folder and its contents.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="model",
    type=AssetTypes.MLFLOW_MODEL,
    name="local-mlflow-diabetes",
    description="Model created from local file.",
)
model = ml_client.models.create_or_update(file_model)

## Build the pipeline

To create the responsible AI dashboard, you'll create a pipeline using the prebuilt components. You can choose which components to use, and which features you want to include in your dashboard. You'll create a dashboard that includes error analysis and model interpretability. 

Next to the responsible AI features, a pipeline to build a dashboard needs to include a component at the start to construct the dashboard, and a component at the end to gather all generated insights.

In [ ]:
model_name = model.name
expected_model_id = f"{model_name}:1"
azureml_model_id = f"azureml:{expected_model_id}"

In [ ]:
label = "latest"

rai_constructor_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_constructor", label=label
)

# we get latest version and use the same version for all components
version = rai_constructor_component.version
print("The current version of RAI built-in components is: " + version)

rai_erroranalysis_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_erroranalysis", version=version
)

rai_explanation_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_explanation", version=version
)

rai_gather_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_gather", version=version
)

When you've retrieved all components you want to use, you can build the pipeline and connect the components in the appropriate order:

1. Construct the dashboard.
1. Add error analysis.
1. Add explanations.
1. Gather all insights and visualize them in the dashboard.

In [ ]:
from azure.ai.ml import Input, dsl
from azure.ai.ml.constants import AssetTypes

compute_name = "aml-cluster"

@dsl.pipeline(
    compute=compute_name,
    description="RAI insights on diabetes data",
    experiment_name=f"RAI_insights_{model_name}",
)
def rai_decision_pipeline(
    target_column_name, train_data, test_data
):
    # Initiate the RAIInsights
    create_rai_job = rai_constructor_component(
        title="RAI dashboard diabetes",
        task_type="classification",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        train_dataset=train_data,
        test_dataset=test_data,
        target_column_name=target_column_name,
    )
    create_rai_job.set_limits(timeout=30)

    # Add error analysis
    error_job = rai_erroranalysis_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
    )
    error_job.set_limits(timeout=10)

    # Add explanations
    explanation_job = rai_explanation_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        comment="add explanation", 
    )
    explanation_job.set_limits(timeout=10)

    # Combine everything
    rai_gather_job = rai_gather_component(
        constructor=create_rai_job.outputs.rai_insights_dashboard,
        insight_3=error_job.outputs.error_analysis,
        insight_4=explanation_job.outputs.explanation,
    )
    rai_gather_job.set_limits(timeout=10)

    rai_gather_job.outputs.dashboard.mode = "upload"

    return {
        "dashboard": rai_gather_job.outputs.dashboard,
    }

Now the pipeline has been built, you need to define the two necessary inputs: the training and test dataset.

In [ ]:
from azure.ai.ml import Input
target_feature = "Diabetic"

diabetes_train_pq = Input(
    type="mltable",
    path=f"azureml:{input_train_data}:{data_version}",
    mode="download",
)
diabetes_test_pq = Input(
    type="mltable",
    path=f"azureml:{input_test_data}:{data_version}",
    mode="download",
)

Finally, we'll put everything together: assign the inputs to the pipeline and set the target column (the predicted label).

In [ ]:
import uuid
from azure.ai.ml import Output

# Pipeline to construct the RAI Insights
insights_pipeline_job = rai_decision_pipeline(
    target_column_name="Diabetic",
    train_data=diabetes_train_pq,
    test_data=diabetes_test_pq,
)

# Workaround to enable the download
rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)


## Run the pipeline

When you've successfully built the pipeline, you can submit it. The following code will submit the pipeline and check the status of the pipeline. You can also view the pipeline's status in the studio. 

In [ ]:
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display
import time

def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job


# This is the actual submission
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

When the pipeline is completed, you can review the dashboard in the studio. 